# **ETL PROCESS**

## Objectives

* Fetch data and save it as raw data file and upload it to the workspace. Take the data through the ETL process to clean it and also include a scaled and encoded file for machine learning purposes.

## Inputs

* Raw CSV data file.

## Outputs

* This notebook will hope to generate a clean CSV file of the data as well as a scaled and encoded txt file. 

## Additional Comments

* This dataset was sourced from Kaggle and contains data regarding air pollution quality in the US in from 2000 to 2016.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

Section 1 content

In [1]:
# Import all necessary packages
import numpy as np
import pandas as pd
import sklearn as sk
import feature_engine as fe 
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

print("All packages imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sk.__version__}")
print(f"Feature-engine version: {fe.__version__}")

All packages imported successfully!
NumPy version: 1.26.1
Pandas version: 2.1.1
Scikit-learn version: 1.3.1
Feature-engine version: 1.6.1


In [ ]:
# Read data and return full DataFrame with shape to make sure everything is working
df = pd.read_csv("pollution_us_2000_2016.csv")
print("Data loaded successfully!")
print(f"DataFrame shape: {df.shape}")
df

In [ ]:
# Return the first few values of the DataFrame
df.head()

In [ ]:
# Extract a random, fractioned sample of the data for analysis purposes
df1 = df.sample(frac=0.5, random_state=10)

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Conclusion

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
